In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import datetime
import matplotlib.dates as mdates
from bs4 import BeautifulSoup
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
from statsmodels.tsa.seasonal import STL, seasonal_decompose, MSTL
from matplotlib.ticker import FuncFormatter

/home/miguel/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
